In [1]:
#GLOBAL IMPORTS

import numpy as np
import os
import pandas as pd
import re
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf
#LOCAL IMPORTS
from workspace_utils import active_session
if tf.test.is_gpu_available:
    print ("GPU")

#GLOBAL VARIABLES


Using TensorFlow backend.


GPU


In [2]:
def read_data():
    #Fetch THE DATA
    path_to_data = "ara.txt"

    #Retrive some of the data
    lines = pd.read_table(path_to_data, names = ['source', 'target', 'comments'])
    return lines

lines = read_data()
lines.sample(6)


,source,target,comments
6696,Tom knows the city very well.,يعرف توم المدينة جيدا,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
3140,Don't worry about it!,لا تقلق بشأن ذلك!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1614,No one helped me.,لم يساعدني أحد.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1406,"Are you new, too?",هل أنت جديد أيضاً؟,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
6968,My hands and legs are swollen.,تورمت قدميّ و يديّ.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
8469,Would you mind shutting the door?,أيمكنك أن تقفل الباب؟,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [3]:
'''
    Cleaning the data:
        - Convert english (source) into lowercase
        - Remove Qoutes
        - Remove all special characters like “@, !, *, $, #, ?, %, etc.”
        - Remove numbers .. since they are different!
        - Remove spaces
'''
#Read the data
lines = read_data()
#Shuffle the data
lines = shuffle(lines)

#LOWER CASE
lines.source = lines.source.apply (lambda x: x.lower())
lines.target = lines.target.apply (lambda x: x.lower()) 

#Qoutes
lines.source = lines.source.apply (lambda x: re.sub("'", '', x))
lines.target = lines.target.apply (lambda x: re.sub("'", '', x))

#A list of all punctuations
punc = set(string.punctuation)

punc.add ('؟') #ara
punc.add ('...') #ara
punc.add ('...') #eng
punc.add ('،') #ara
lines.source = lines.source.apply(lambda x: ''.join(char1 for char1 in x if char1 not in punc))
lines.target = lines.target.apply(lambda x: ''.join(char1 for char1 in x if char1 not in punc))

#Remving digits
num_digits= str.maketrans('','', digits)
lines.source = lines.source.apply(lambda x: x.translate(num_digits))
lines.target = lines.target.apply(lambda x: x.translate(num_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))

#adding start/end tags
lines.source = lines.source.apply(lambda x : '<sos> '+ x + ' <eos>')
lines.target = lines.target.apply(lambda x : '<sos> '+ x + ' <eos>')
lines.sample(6)


,source,target,comments
1791,<sos> your dog is here <eos>,<sos> هنا كلبك <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
5063,<sos> im glad to see you back <eos>,<sos> سررت بعودتك <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
7780,<sos> im very grateful for your help <eos>,<sos> أنا ممتن للغاية لمساعدتك <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
8041,<sos> why couldnt you come yesterday <eos>,<sos> لماذا لم تستطع القدوم بالأمس <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
138,<sos> i am cold <eos>,<sos> أشعر بالبرد <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4758,<sos> tom almost convinced me <eos>,<sos> أقنعني توم تقريباً <eos>,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [4]:
#The whole Source set length

source_vocab = set()
for line in lines.source:
    chars = line.split(' ')
    for char in chars: 
        if char not in source_vocab:
            source_vocab.add(char)
            
target_vocab = set()
for line in lines.target:
    chars = line.split(' ')
    for char in chars: 
        if char not in target_vocab:
            target_vocab.add(char)
        
print (f'Uniques in the Source count \t{len(source_vocab)}')
print ()
print (f'Uniques in the Target count \t{len(target_vocab)}')
#NOTE: the arabic language is vastly rich!

source_vocab = sorted(list(source_vocab))
target_vocab = sorted(list(target_vocab))

# Input tokens for encoder
num_encoder_tokens=len(source_vocab)
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_vocab) + 1


Uniques in the Source count 	4168

Uniques in the Target count 	11774


In [5]:
#Max lengths for Source and Targets
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
src_max_length= max(source_length_list)

target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
trg_max_length= max(target_length_list)

print (f'Max length of Source lang: {src_max_length}')
print (f'Max length of Target lang: {trg_max_length}')

#Word to index dictionary
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_vocab)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_vocab)])

#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

#Shuffle the data


Max length of Source lang: 36
Max length of Target lang: 38


In [6]:
#Splitting the data
#train_test_split from Sklearn lib

X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
print (f'Source Training data shape: \t{X_train.shape}')
print (f'Target Training data shape: \t{y_train.shape}')
print ()
print (f'Source Test data shape: \t{X_test.shape}')
print (f'Target Test data shape: \t{y_test.shape}')


Source Training data shape: 	(10301,)
Target Training data shape: 	(10301,)

Source Test data shape: 	(1145,)
Target Test data shape: 	(1145,)


In [7]:
#generate_batch() will provide batches of data for fit_generator()

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, src_max_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, trg_max_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, trg_max_length, num_decoder_tokens),dtype='float32')
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split(' ')):
                    encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split(' ')):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [8]:
#MODEL

#Essenteal parameters
train_samples = len(X_train)
test_samples = len(X_test)
batch_size = 128
epochs = 65
latent_dim=256

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#optimizer: rmsprop .. try adam!
#loss: cross entropy
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


In [9]:
with active_session():
    # do long-running work here
    model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                        steps_per_epoch = train_samples//batch_size,
                        epochs=epochs,
                        validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                        validation_steps = test_samples//batch_size)

Epoch 1/100
80/80 [==============================] - 65s 813ms/step - loss: 6.9097 - acc: 0.1959 - val_loss: 6.6283 - val_acc: 0.2061
Epoch 2/100
80/80 [==============================] - 62s 774ms/step - loss: 6.3271 - acc: 0.2098 - val_loss: 6.6216 - val_acc: 0.2164
Epoch 3/100
80/80 [==============================] - 62s 770ms/step - loss: 6.0551 - acc: 0.2196 - val_loss: 6.5504 - val_acc: 0.2271
Epoch 4/100
80/80 [==============================] - 62s 775ms/step - loss: 5.8104 - acc: 0.2268 - val_loss: 6.6461 - val_acc: 0.2268
Epoch 5/100
80/80 [==============================] - 60s 755ms/step - loss: 5.5926 - acc: 0.2339 - val_loss: 6.5955 - val_acc: 0.2254
Epoch 6/100
80/80 [==============================] - 62s 777ms/step - loss: 5.3885 - acc: 0.2395 - val_loss: 6.5407 - val_acc: 0.2352
Epoch 7/100
80/80 [==============================] - 61s 757ms/step - loss: 5.1869 - acc: 0.2470 - val_loss: 6.4970 - val_acc: 0.2408
Epoch 8/100
80/80 [==============================] - 61s 763ms

Epoch 62/100
80/80 [==============================] - 61s 764ms/step - loss: 0.0813 - acc: 0.9910 - val_loss: 6.3922 - val_acc: 0.3299
Epoch 63/100
80/80 [==============================] - 61s 767ms/step - loss: 0.0742 - acc: 0.9910 - val_loss: 6.4232 - val_acc: 0.3305
Epoch 64/100
80/80 [==============================] - 61s 760ms/step - loss: 0.0715 - acc: 0.9913 - val_loss: 6.4412 - val_acc: 0.3333
Epoch 65/100
80/80 [==============================] - 61s 760ms/step - loss: 0.0610 - acc: 0.9924 - val_loss: 6.4364 - val_acc: 0.3384
Epoch 66/100
80/80 [==============================] - 61s 759ms/step - loss: 0.0601 - acc: 0.9927 - val_loss: 6.4606 - val_acc: 0.3356
Epoch 67/100
80/80 [==============================] - 61s 766ms/step - loss: 0.0566 - acc: 0.9926 - val_loss: 6.4754 - val_acc: 0.3354
Epoch 68/100
80/80 [==============================] - 60s 756ms/step - loss: 0.0525 - acc: 0.9927 - val_loss: 6.4801 - val_acc: 0.3334
Epoch 69/100
80/80 [==============================] - 6

In [10]:
#Saving the model
model.save_weights('nmt_weights_100epochs.h5')

In [11]:
#Load the model info
model.load_weights('nmt_weights_100epochs.h5')

In [12]:
# Encode the input sequence to get the "Context vectors"

#CREATE THE ENCODER
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['<sos>']
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [20]:
#predicting from training set and test set
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])
print ('\n\n')
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=100
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])



Input Source sentence: <sos> they hated tom <eos>
Actual Target Translation: كانوا يكرهون توم <
Predicted Target Translation:  كانوا يكرهون توم <



Input Source sentence: <sos> you never listen i might as well talk to the wall <eos>
Actual Target Translation: لا تُنْصِتُ أبدًا ربما يَحْسُنُ بي التحدث إلى الجدار <
Predicted Target Translation:  هذا تخصصه <
